In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn import metrics
from sklearn.neural_network import MLPClassifier

import pickle
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox

vectorizer = CountVectorizer() 

In [2]:
news1 = pd.read_csv("1-8000.csv")
neg1 = news1[news1.label == 'neg']
pos1 = news1[news1.label == 'pos']

news2 = pd.read_csv("8001-end.csv")
neg2 = news2[news2.mlabel == 'neg']
pos2 = news2[news2.mlabel == 'pos']

news3 = pd.read_csv("all_final.csv")
neg3 = news3[news3.V2 == 'neg']
pos3 = news3[news3.V2 == 'pos']
neut=news3[news3.V2 == 'neut']

negative = list(neg1.content) + list(neg2.content) + list(neg3.V1)
positive = list(pos1.content) + list(pos2.content) + list(pos3.V1)

mylist1 = list(dict.fromkeys(negative))
mylist2 = list(dict.fromkeys(positive))

neglabels = ['negative' for l in range(len(mylist1))]
poslabels = ['positive' for l in range(len(mylist2))]

news = mylist1 + mylist2
labels = neglabels + poslabels

news=np.array(news)
labels=np.array(labels)

print(type(news))
print(type(labels))
train_corpus, test_corpus, train_labels, test_labels = train_test_split(news, labels,test_size=0.2)

labels.reshape(-1,1)

C:\Users\Owner\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


array([['negative'],
       ['negative'],
       ['negative'],
       ...,
       ['positive'],
       ['positive'],
       ['positive']], dtype='<U8')

In [3]:
vectorized_train = vectorizer.fit_transform(train_corpus)
vectorized_test = vectorizer.transform(test_corpus)

In [4]:
vectorized_test.shape

(2485, 94647)

In [5]:
mlp=MLPClassifier()
mlp.fit(vectorized_train,train_labels)

MLPClassifier()

In [6]:
with open('model.pickle', 'wb') as f:
    pickle.dump(mlp, f)
    
with open('vectorizer.pickle', 'wb') as f:
    pickle.dump(vectorizer, f)

In [7]:
with open('model.pickle', 'rb') as f:
    mlp = pickle.load(f)
    
with open('vectorizer.pickle', 'rb') as f:
    vectorizer = pickle.load(f)
    
predictions=mlp.predict(vectorized_test)
l = sorted(set(labels))
print('Accuracy')
accuracy = np.sum(predictions == test_labels) / len(test_labels)
print(accuracy)
print('Confusion Matrix and Classification Report')
print(metrics.classification_report(test_labels, predictions, target_names=l))
print(metrics.confusion_matrix(test_labels, predictions, labels=l))

test1 = 'arının qaliblərinin mükafatlandırılması mərasimi keçirilib. Milli.Az Trend-ə istinadən bildirir ki, kişilər inci yeri Azərbaycan təmsilçisi Mixail Malkin, üçüncü yeri isə digər rusiyalı idmançı Maksim Ryabikov tutub. Qadınlar arasında qızıl və gümüş medallara ukraynalı gimnastlar Darina Kozyarska və Alina Mamçur sahib olublar. Həmçinin turnirdə ən yüksək icra xalını toplamış gimnastlara ənənəvi "AGF Trophy" Kuboku təqdim olunub. Bu mükafata Azərbaycan idmançısı Mixail Malkin və Çin təmsilçiləri Qianqi Lin və Syao Tu layiq görülüblər.'
test2 = 'Məktəb direktorunu döyən "Drakon" ləqəbli aktyor Rza Rzayevə cinayət işi açılıb. Milli.Az musavat.com-a istinadən bildirir ki, Rza Rzayevə Nizami rayon Polis İdarəsinin İstintaq şöbəsində aparılan araşdırmadan sonra Cinayət Məcəlləsinin 221.3-cü (Xuliqanlıq, silahdan və ya silah qismində istifadə edilən əşyaları tətbiq etməklə törədildikdə) maddəsi ilə ittiham irəli sürülüb. Təqsirləndirilən şəxsin əməli sübuta yetirilərsə, onu üç ildən yeddi ilədək müddətə azadlıqdan məhrum etmə cəzası gözləyir. İş baxılması üçün aidiyyatı üzrə məhkəməyə göndərilib.'
test3 = 'Usman Kuçkar dünyasını dəyişdi,"  Tanınmış şair və tərcüməçi, Azərbaycan ədəbiyyatının yaxın dostu Usman Kuçkar dünyasını dəyişdi.O Azərbaycan ədəbiyyatinin ən yaxşı nümunələrinin tərcüməçisi kimi tanınırdı . Allah rəhmət eləsin'

print(mlp.predict(vectorizer.transform([test1,test2,test3])))

Accuracy
0.9484909456740442
Confusion Matrix and Classification Report
              precision    recall  f1-score   support

    negative       0.96      0.96      0.96      1555
    positive       0.93      0.93      0.93       930

    accuracy                           0.95      2485
   macro avg       0.95      0.94      0.94      2485
weighted avg       0.95      0.95      0.95      2485

[[1492   63]
 [  65  865]]
['positive' 'negative' 'negative']


In [8]:
class SentimentApp(tk.Frame):
    APP_TITLE = "Sentiment App"
    ICON_FILE = "question.ico"
    PACK_FILL_EXPAND = {"expand": True, "fill": tk.BOTH}
    PLACE_MIDDLE = {"anchor": tk.N, "relx": 0.5}
    TRANSLATE = {"positive": "pozitiv", "negative": "neqativ"}
    TEXTFONT = ("Arial", 12)
    APP_BACKGROUND = "#6D6F90"
    HEADER_TEXT = "Aşağıdakı qutuya mətn girin və onun pozitiv və ya neqativ olduğunu öyrənin!"

    def __init__(self, master):
        super().__init__(master)
        self.master = master
        self.pack(**SentimentApp.PACK_FILL_EXPAND)
        self.configure_master()
        self.configure_styling()
        self.create_widgets()

    def configure_master(self):
        self.master.title(SentimentApp.APP_TITLE)
        self.master.iconbitmap(SentimentApp.ICON_FILE)
        self.master.geometry("600x600")
        self.master.resizable(False, False)

    def configure_styling(self):
        self.style = ttk.Style()
        self.style.theme_use("clam")
        self.style.configure("TButton", font=SentimentApp.TEXTFONT)
        self.style.configure("TLabel", font=SentimentApp.TEXTFONT, background=SentimentApp.APP_BACKGROUND)

    def create_widgets(self):
        self.mainframe = tk.LabelFrame(self, bg=SentimentApp.APP_BACKGROUND)
        self.mainframe.place(relx=0, rely=0, relheight=1, relwidth=1)

        self.header = ttk.Label(self.mainframe, text=SentimentApp.HEADER_TEXT)
        self.header.place(**SentimentApp.PLACE_MIDDLE, rely=0.2)

        self.sc_y = ttk.Scrollbar(self.mainframe, orient=tk.VERTICAL)
        self.text_input = tk.Text(self.mainframe, width=52, height=10, yscrollcommand=self.sc_y.set, font=SentimentApp.TEXTFONT)
        self.sc_y.config(command=self.text_input.yview)
        self.sc_y.place(relx=0.8984, rely=0.40016, relheight=0.308)
        self.text_input.place(**SentimentApp.PLACE_MIDDLE, rely=0.4)

        self.submit_button = self.label_button = ttk.Button(self.mainframe, takefocus=0, text="Submit",
                                                            command=self.determine_text_sentiment)
        self.submit_button.place(**SentimentApp.PLACE_MIDDLE, rely=0.8)

    def determine_text_sentiment(self):
        user_input = self.text_input.get("1.0", tk.END)
        with open('model.pickle', 'rb') as f:
            mlp = pickle.load(f)
        result = mlp.predict(vectorizer.transform([user_input]))
        messagebox.showinfo("Nəticə", f"Sizin xəbərinizin tipi: {SentimentApp.TRANSLATE[result[0]]}")


def main():
    root = tk.Tk()
    app = SentimentApp(root)
    app.mainloop()


if __name__ == "__main__":
    main()

Neqativ xəbər nümunəsi:

Bu gün Tbilisinin Lilo qəsəbəsində yaşayış binasında partlayış olub.

APA-nın “Mtavari” telekanalına istinadən verdiyi məlumata görə, Tenqiz küçəsindəki binada baş verən partlayış nəticəsində 65 yaşlı qadın ölüb.

Binanın dördüncü mərtəbəsindəki mənzildə baş verən partlayış zamanı partlayış dalğası eyvanın qapısını çıxarıb, evin sahibi dördüncü mərtəbədən yerə düşüb O, aldığı travmalardan hadisə yerində ölüb.

Pozitiv xəbər nümunəsi:

Azərbaycan millisinin futbolçusu Ramil Şeydayev uzun fasilədən sonra Premyer Liqada start heyətində yer alıb.

Apasport.az saytının məlumatına görə, hücumçu “Sabah”ın “Qəbələ” ilə bugünkü qarşılaşmasına “11-lik”də başlayacaq.

XXI turun bu matçı ilə Ramilin start heyəti həsrəti bitəcək. O, sonuncu dəfə 147 gün öncə ilk dəqiqələrdən meydana çıxıb.
